In [1]:
from keras2plc import ST_writer
from keras2plc import nn_reader

folder = "test_ST_files/"
model_name = "Dense_v1"
model_file = "test_model.keras"

reader = nn_reader(model_file,model_name,3,1)
layers_contents = reader.generate_struct_layers()
layersWeights_contents = reader.generate_struct_layersweights()
writer = ST_writer(model_name, layers_contents,layersWeights_contents)
reader.generate_weights_file(folder,True)

c:\Users\ac143081\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 14 variables whereas the saved optimizer has 2 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [2]:
writer.write_ST_files_to(folder, overwrite_if_exists=True)
print(writer.create_example_usage(dims_input=2, dims_output=1))

The following code can be used to call the generated model:
        Assuming declared input/output for model:
        
            input : ARRAY[0..1] OF LREAL;
            result : ARRAY[0..0] OF LREAL;

        Then call as:

            FB_Dense_v1(pointer_input:=ADR(input), pointer_output:=ADS(result), nn:=GVL_Dense_v1.nn);      
        
        
